# Discusión — Capítulo 5

Este notebook presenta una **discusión conceptual y práctica** sobre los ejercicios 4.5.2 y 4.5.3 del proyecto *Quantum Simulator*. Se analizan los conceptos de media, varianza y matrices unitarias, con ejemplos numéricos y conclusiones físicas.

## Celda 1 — Importaciones y clases fallback
Intentamos importar las clases `Observable` y `QuantumDynamics` desde el paquete principal. Si no se encuentran, se usan implementaciones locales equivalentes para garantizar la ejecución.

In [ ]:
# Celda 1: Importaciones
import sys
sys.path.append('..')
import numpy as np

try:
    from quantum_simulator.observables import Observable
    from quantum_simulator.dynamics import QuantumDynamics
    print("Importadas clases desde quantum_simulator.*")
except Exception as e:
    print("No se pudo importar desde quantum_simulator. Usando implementaciones locales. Error:", e)
    class Observable:
        def __init__(self, matrix):
            self.M = np.asarray(matrix, dtype=complex)
        def mean(self, state):
            psi = np.asarray(state, dtype=complex).reshape(-1)
            psi = psi / np.linalg.norm(psi)
            return float(np.vdot(psi, self.M.dot(psi)))
        def variance(self, state):
            psi = np.asarray(state, dtype=complex).reshape(-1)
            psi = psi / np.linalg.norm(psi)
            exp = np.vdot(psi, self.M.dot(psi))
            exp2 = np.vdot(psi, self.M.dot(self.M).dot(psi))
            return float(exp2 - exp**2)
    class QuantumDynamics:
        def __init__(self, initial_state):
            a = np.asarray(initial_state, dtype=complex).reshape(-1)
            self.state = a / np.linalg.norm(a)
        def apply_unitary_evolution(self, unitary_list):
            st = self.state.copy()
            for U in unitary_list:
                st = np.asarray(U, dtype=complex).dot(st)
            self.state = st
            return self.state

## Celda 2 — Ejercicio 4.5.2: Media y Varianza de Observables
**Objetivo:** Calcular la media y varianza de un observable (matriz hermitiana) sobre un estado dado y analizar su interpretación física.

In [ ]:
print("=== Ejemplo 4.5.2 ===")
# Operador de spin en dirección z
sigma_z = np.array([[1, 0], [0, -1]])
observable = Observable(sigma_z)

# Estado |+> = (|0> + |1>)/√2
state_plus = [1/np.sqrt(2), 1/np.sqrt(2)]
mean = observable.mean(state_plus)
variance = observable.variance(state_plus)

print(f"Estado: |+> = [1/√2, 1/√2]")
print(f"Media de σ_z: {mean:.4f}")
print(f"Varianza de σ_z: {variance:.4f}")
print("Interpretación: En el estado |+>, la medición de spin z es incierta (alta varianza).")

## Celda 3 — Ejercicio 4.5.3: Matrices Unitarias y Dinámica Cuántica
**Objetivo:** Demostrar que las matrices unitarias preservan la norma y los productos internos, garantizando una evolución reversible del sistema.

In [ ]:
print("\n=== Ejemplo 4.5.3 ===")
# Matriz de Hadamard
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], 
              [1/np.sqrt(2), -1/np.sqrt(2)]])

# Verificar unitariedad
print(f"H es unitaria? {np.allclose(H @ H.conj().T, np.eye(2))}")

# Evolución de estado inicial |0>
initial_state = [1, 0]
dynamics = QuantumDynamics(initial_state)
final_state = dynamics.apply_unitary_evolution([H])

print(f"Estado inicial: {initial_state}")
print(f"Estado después de H: {final_state}")
print(f"Norma conservada: {np.linalg.norm(initial_state):.4f} = {np.linalg.norm(final_state):.4f}")

## Celda 4 — Conclusiones
1. Los observables hermitianos garantizan valores de medición reales.
2. La varianza mide la incertidumbre cuántica inherente.
3. Las matrices unitarias preservan la norma y la probabilidad.
4. La mecánica cuántica es fundamentalmente **probabilística y reversible**, base de la computación cuántica moderna (Grover, Shor, etc.).